In [834]:
# importing common libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# warnings
import warnings
warnings.filterwarnings('ignore')

In [835]:
# load dataset
path = r'C:\Users\kalema\Desktop\Recess\resources\datasets\MalariaAfricaDataset.csv'
dataset = pd.read_csv(path)
df = pd.DataFrame(dataset)
df.head(10)

,Country Name,Year,Country Code,"Incidence of malaria (per 1,000 population at risk)",Malaria cases reported,Use of insecticide-treated bed nets (% of under-5 population),Children with fever receiving antimalarial drugs (% of children under age 5 with fever),Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women),People using safely managed drinking water services (% of population),"People using safely managed drinking water services, rural (% of rural population)",...,Urban population growth (annual %),People using at least basic drinking water services (% of population),"People using at least basic drinking water services, rural (% of rural population)","People using at least basic drinking water services, urban (% of urban population)",People using at least basic sanitation services (% of population),"People using at least basic sanitation services, rural (% of rural population)","People using at least basic sanitation services, urban (% of urban population)",latitude,longitude,geometry
0,Algeria,2007,DZA,0.01,26.0,NaN,NaN,NaN,NaN,NaN,...,2.71,91.68,85.83,94.78,85.85,76.94,90.57,28.033886,1.659626,POINT (28.033886 1.659626)
1,Angola,2007,AGO,286.72,1533485.0,18.0,29.8,1.5,NaN,NaN,...,5.01,47.96,23.77,65.83,37.26,14.00,54.44,-11.202692,17.873887,POINT (-11.202692 17.873887)
2,Benin,2007,BEN,480.24,0.0,NaN,NaN,NaN,NaN,NaN,...,4.09,63.78,54.92,76.24,11.80,4.29,22.36,9.307690,2.315834,POINT (9.307689999999999 2.315834)
3,Botswana,2007,BWA,1.03,390.0,NaN,NaN,NaN,NaN,NaN,...,4.80,78.89,57.60,94.35,61.60,39.99,77.30,-22.328474,24.684866,POINT (-22.328474 24.684866)
4,Burkina Faso,2007,BFA,503.80,44246.0,NaN,NaN,NaN,NaN,NaN,...,5.91,52.27,45.13,76.15,15.60,6.38,46.49,12.238333,-1.561593,POINT (12.238333 -1.561593)
5,Burundi,2007,BDI,222.92,1101644.0,NaN,NaN,NaN,NaN,NaN,...,5.82,54.73,51.39,85.24,45.91,46.26,42.78,-3.373056,29.918886,POINT (-3.373056 29.918886)
6,Cabo Verde,2007,CPV,0.15,18.0,NaN,NaN,NaN,NaN,NaN,...,2.66,81.39,71.02,88.48,54.85,38.90,65.77,16.002082,-24.013197,POINT (16.002082 -24.013197)
7,Cameroon,2007,CMR,386.68,0.0,NaN,NaN,NaN,NaN,NaN,...,3.96,60.09,38.85,81.55,39.25,23.01,55.66,7.369722,12.354722,POINT (7.369722 12.354722)
8,Central African Republic,2007,CAF,471.11,0.0,NaN,NaN,NaN,NaN,NaN,...,2.30,51.10,38.18,71.87,20.55,8.76,39.50,6.611111,20.939444,POINT (6.611111 20.939444)
9,Chad,2007,TCD,210.71,48288.0,NaN,NaN,NaN,NaN,NaN,...,3.55,40.34,30.88,74.12,11.15,5.75,30.45,15.454166,18.732207,POINT (15.454166 18.732207)


In [836]:
# number of rows and columns
number_of_columns = df.shape[1]
number_of_rows = df.shape[0]
print(f'Number of columns: {number_of_columns}')
print(f'Number of rows: {number_of_rows}')

Number of columns: 27
Number of rows: 594


In [837]:
# Column names
df.columns

Index(['Country Name', 'Year', 'Country Code',
       'Incidence of malaria (per 1,000 population at risk)',
       'Malaria cases reported',
       'Use of insecticide-treated bed nets (% of under-5 population)',
       'Children with fever receiving antimalarial drugs (% of children under age 5 with fever)',
       'Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women)',
       'People using safely managed drinking water services (% of population)',
       'People using safely managed drinking water services, rural (% of rural population)',
       'People using safely managed drinking water services, urban (% of urban population)',
       'People using safely managed sanitation services (% of population)',
       'People using safely managed sanitation services, rural (% of rural population)',
       'People using safely managed sanitation services, urban  (% of urban population)',
       'Rural population (% of total population)',
       'Rural popula

In [838]:
# column datatypes
df.dtypes

Country Name                                                                                object
Year                                                                                         int64
Country Code                                                                                object
Incidence of malaria (per 1,000 population at risk)                                        float64
Malaria cases reported                                                                     float64
Use of insecticide-treated bed nets (% of under-5 population)                              float64
Children with fever receiving antimalarial drugs (% of children under age 5 with fever)    float64
Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women)      float64
People using safely managed drinking water services (% of population)                      float64
People using safely managed drinking water services, rural (% of rural population)         float64
People usi

In [839]:
# Converting year datatype to datetime
df['Year'] = pd.to_datetime(df.Year,format='%Y').dt.year
df.dtypes

Country Name                                                                                object
Year                                                                                         int64
Country Code                                                                                object
Incidence of malaria (per 1,000 population at risk)                                        float64
Malaria cases reported                                                                     float64
Use of insecticide-treated bed nets (% of under-5 population)                              float64
Children with fever receiving antimalarial drugs (% of children under age 5 with fever)    float64
Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women)      float64
People using safely managed drinking water services (% of population)                      float64
People using safely managed drinking water services, rural (% of rural population)         float64
People usi

In [840]:
# Renaming columns
new_column_names = {
    'Country Name': 'Country',
    'Year': 'Year',
    'Country code': 'Country_code',
    'Malaria cases reported': 'Reported_cases',
    'Incidence of malaria (per 1,000 population at risk)': 'Incidences_per_1000_population_at_risk',
    'Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women)': '%_of_pregnant_women_using_IPT',
    'Use of insecticide-treated bed nets (% of under-5 population)': '%_using_IBNs',
    'Children with fever receiving antimalarial drugs (% of children under age 5 with fever)': '%_of_children_under_age_5_with_fever',
    'Rural population (% of total population)': '%_of_rural_population',
    'Urban population (% of total population)': '%_of_urban_population',
    'Rural population growth (annual %)': 'annual_%_growth_of_rural_population',
    'Urban population growth (annual %)': 'annual_%_growth_of_urban_population',
    'People using safely managed sanitation services (% of population)': '%_using_safe_sanity_services',
    'People using safely managed sanitation services, urban  (% of urban population)': '%_of_urban_using_safe_sanity_services',
    'People using safely managed sanitation services, rural (% of rural population)': '%_of_rural_using_safe_sanity_services',
    'People using at least basic sanitation services (% of population)': '%_using_atleast_basic_sanity_services',
    'People using at least basic sanitation services, urban  (% of urban population)': '%_of_urban_using_atleast_basic_sanity_services',
    'People using at least basic sanitation services, rural (% of rural population)': '%_of_rural_using_atleast_basic_sanity_services',
    'People using at least basic drinking water services (% of population)': '%_using_atleast_basic_drinking_water_services',
    'People using at least basic drinking water services, urban (% of urban population)': '%_of_urban_using_atleast_basic_drinking_water_services',
    'People using at least basic drinking water services, rural (% of rural population)': '%_of_rural_using_atleast_basic_drinking_water_services',
    'People using safely managed drinking water services (% of population)': '%_using_safe_drinking_water_services',
    'People using safely managed drinking water services, urban (% of urban population)': '%_of_urban_using_safe_drinking_water_services',
    'People using safely managed drinking water services, rural (% of rural population)': '%_of_rural_using_safe_drinking_water_services',
    'longitude': 'longitude',
    'latitude': 'latitude',
    'geometry': 'geometry'
}

df.rename(columns=new_column_names, inplace=True)


In [841]:
# Review changes
df.columns

Index(['Country', 'Year', 'Country Code',
       'Incidences_per_1000_population_at_risk', 'Reported_cases',
       '%_using_IBNs', '%_of_children_under_age_5_with_fever',
       '%_of_pregnant_women_using_IPT', '%_using_safe_drinking_water_services',
       '%_of_rural_using_safe_drinking_water_services',
       '%_of_urban_using_safe_drinking_water_services',
       '%_using_safe_sanity_services', '%_of_rural_using_safe_sanity_services',
       '%_of_urban_using_safe_sanity_services', '%_of_rural_population',
       'annual_%_growth_of_rural_population', '%_of_urban_population',
       'annual_%_growth_of_urban_population',
       '%_using_atleast_basic_drinking_water_services',
       '%_of_rural_using_atleast_basic_drinking_water_services',
       '%_of_urban_using_atleast_basic_drinking_water_services',
       '%_using_atleast_basic_sanity_services',
       '%_of_rural_using_atleast_basic_sanity_services',
       '%_of_urban_using_atleast_basic_sanity_services', 'latitude',
      

In [842]:
# number of missing values per column in descending order
number_of_missing_values = df.isnull().sum().sort_values(ascending=False)
number_of_missing_values

%_of_rural_using_safe_drinking_water_services             506
%_using_safe_drinking_water_services                      495
%_of_pregnant_women_using_IPT                             488
%_of_rural_using_safe_sanity_services                     484
%_of_children_under_age_5_with_fever                      472
%_of_urban_using_safe_sanity_services                     462
%_using_safe_sanity_services                              462
%_using_IBNs                                              462
%_of_urban_using_safe_drinking_water_services             418
Reported_cases                                             44
Incidences_per_1000_population_at_risk                     44
%_of_urban_using_atleast_basic_sanity_services             28
%_of_rural_using_atleast_basic_drinking_water_services     28
%_of_urban_using_atleast_basic_drinking_water_services     28
%_of_rural_using_atleast_basic_sanity_services             28
%_of_rural_population                                       6
annual_%

In [843]:
# Percentage of null values per column in descending order
percentage_of_missing_values = (df.isnull().sum()/df.shape[0]*100).sort_values(ascending=False)
percentage_of_missing_values = percentage_of_missing_values.apply(lambda x: f'{x:.2f}%')
percentage_of_missing_values

%_of_rural_using_safe_drinking_water_services             85.19%
%_using_safe_drinking_water_services                      83.33%
%_of_pregnant_women_using_IPT                             82.15%
%_of_rural_using_safe_sanity_services                     81.48%
%_of_children_under_age_5_with_fever                      79.46%
%_of_urban_using_safe_sanity_services                     77.78%
%_using_safe_sanity_services                              77.78%
%_using_IBNs                                              77.78%
%_of_urban_using_safe_drinking_water_services             70.37%
Reported_cases                                             7.41%
Incidences_per_1000_population_at_risk                     7.41%
%_of_urban_using_atleast_basic_sanity_services             4.71%
%_of_rural_using_atleast_basic_drinking_water_services     4.71%
%_of_urban_using_atleast_basic_drinking_water_services     4.71%
%_of_rural_using_atleast_basic_sanity_services             4.71%
%_of_rural_population    

Checking for rows where the sum of '%_of_rural_population' and '%_of_urban_population' is not equal to 100

In [844]:
# Obtain sum of %_of_rural_population and %_of_urban_population
df['%_of_total_population'] = df[['%_of_rural_population', '%_of_urban_population']].sum(axis=1)
# Checking for rows where %_of_rural_population + %_of_urban_population is not equal to 100
invalid_total_population = df[(df['%_of_rural_population'].notnull()) & (df['%_of_urban_population'].notnull()) &
                             (df['%_of_rural_population'] + df['%_of_urban_population'] != 100)].reset_index(drop=True)

print(f'Number of such rows: {invalid_total_population.shape[0]}')
    
invalid_total_population[['Country', 'Year', '%_of_rural_population', '%_of_urban_population', '%_of_total_population']]

Number of such rows: 45


,Country,Year,%_of_rural_population,%_of_urban_population,%_of_total_population
0,Malawi,2007,84.77,15.24,100.01
1,Senegal,2007,57.47,42.54,100.01
2,Seychelles,2007,47.72,52.29,100.01
3,Equatorial Guinea,2008,37.26,62.75,100.01
4,Eswatini,2008,77.96,22.05,100.01
5,Guinea-Bissau,2008,60.68,39.33,100.01
6,Rwanda,2008,83.08,16.93,100.01
7,South Sudan,2008,82.45,17.56,100.01
8,Gabon,2009,15.05,84.96,100.01
9,Liberia,2009,52.56,47.45,100.01


Algorithm to normalize the '%_of_rural_population' and '%_of_urban_population' columns for rows where the sum is not 100.

In [845]:
# Algorithm to normalize %_of_rural_population and %_of_urban_population where the total population percentage is not 100
def normalize_percentages(df):
    for index, row in df.iterrows():
        if pd.notnull(row['%_of_rural_population']) and pd.notnull(row['%_of_urban_population']) and \
                (row['%_of_rural_population'] + row['%_of_urban_population'] != 100):
            # Calculate normalized values
            normalized_rural_percentage = (row['%_of_rural_population'] / (row['%_of_rural_population'] + row['%_of_urban_population'])) * 100
            normalized_urban_percentage = (row['%_of_urban_population'] / (row['%_of_rural_population'] + row['%_of_urban_population'])) * 100
            # Round to 2 decimal places
            normalized_rural_percentage = round(normalized_rural_percentage, 2)
            normalized_urban_percentage = round(normalized_urban_percentage, 2)
            # Assign the values to new columns
            df.at[index, '%_of_rural_population'] = normalized_rural_percentage
            df.at[index, '%_of_urban_population'] = normalized_urban_percentage
    return df

df = normalize_percentages(df)

Review normalization results

In [846]:
# Checking for rows where %_of_rural_population + %_of_urban_population is not equal to 100
new_invalid_total_population = df[(df['%_of_rural_population'].notnull()) & (df['%_of_urban_population'].notnull()) &
                             (df['%_of_rural_population'] + df['%_of_urban_population'] != 100)].reset_index(drop=True)

print(f'Number of such rows: {new_invalid_total_population.shape[0]}')
    
new_invalid_total_population[['Country', 'Year', '%_of_rural_population', '%_of_urban_population', '%_of_total_population']]

Number of such rows: 0


,Country,Year,%_of_rural_population,%_of_urban_population,%_of_total_population


Fill cells where values are "Missing At Random", which means that their values can be determined using values from other columns.

The algorithm is based off of four equations which represent a mathematical relationship between some of the columns as shown below.
1. %_using_safe_sanity_services = (%_of_rural_using_safe_sanity_services * %_of_rural_population) + (%_of_urban_using_safe_sanity_services * %_of_urban_population)/100

2. %_using_atleast_basic_drinking_water_services = (%_of_rural_using_atleast_basic_drinking_water_services * %_of_rural_population) + (%_of_urban_using_atleast_basic_drinking_water_services * %_of_urban_population)/100

3. %_using_atleast_basic_sanity_services = (%_of_rural_using_atleast_basic_sanity_services * %_of_rural_population) + (%_of_urban_using_atleast_basic_sanity_services * %_of_urban_population)/100

4. %_using_safe_drinking_water_services = (%_of_rural_using_safe_drinking_water_services * %_of_rural_population) + (%_of_urban_using_safe_drinking_water_services * %_of_urban_population)/100

Eritrea is the only country with null values in the '%_of_rural_population' and '%_of_urban_population' columns, which can be calculated using values provided in the columns '%_using_atleast_basic_sanity_services', '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', and '%_of_urban_using_atleast_basic_drinking_water_services'

Eritrea is also the only contry with null values in the 'annual_&#37;\_growth_of_rural_population' and 'annual\_&#37;_growth_of_urban_population' columns
 

In [847]:
# Obtain rows where the country is Eritrea
eritrea = df[df['Country'] == 'Eritrea'].reset_index(drop=True)
# Display population percentages
eritrea[['Country', 'Year', '%_of_rural_population', '%_of_urban_population', 'annual_%_growth_of_rural_population', 'annual_%_growth_of_urban_population']]


,Country,Year,%_of_rural_population,%_of_urban_population,annual_%_growth_of_rural_population,annual_%_growth_of_urban_population
0,Eritrea,2007,66.98,33.02,1.22,5.62
1,Eritrea,2008,66.00,34.00,0.71,5.12
2,Eritrea,2009,65.41,34.59,0.96,3.55
3,Eritrea,2010,64.82,35.18,0.70,3.30
4,Eritrea,2011,64.23,35.77,0.44,3.04
5,Eritrea,2012,NaN,NaN,NaN,NaN
6,Eritrea,2013,NaN,NaN,NaN,NaN
7,Eritrea,2014,NaN,NaN,NaN,NaN
8,Eritrea,2015,NaN,NaN,NaN,NaN
9,Eritrea,2016,NaN,NaN,NaN,NaN


Algorithm to fill the %_of_rural_population and %_of_urban_population columns.

In [848]:

def fill_missing_percentage_population(df):
    for index, row in df.iterrows():
        if pd.notnull(row['%_using_atleast_basic_drinking_water_services']) and pd.notnull(row['%_of_rural_using_atleast_basic_drinking_water_services']) \
            and pd.notnull(row['%_of_urban_using_atleast_basic_drinking_water_services']) and pd.notnull(row['%_using_atleast_basic_sanity_services']) \
            and pd.notnull(row['%_of_rural_using_atleast_basic_sanity_services']) and pd.notnull(row['%_of_urban_using_atleast_basic_sanity_services']) and \
            pd.isnull(row['%_of_rural_population']) and pd.isnull(row['%_of_urban_population']):

            coefficients = [[row['%_of_rural_using_atleast_basic_drinking_water_services']/100, row['%_of_urban_using_atleast_basic_drinking_water_services']/100],
                                     [row['%_of_rural_using_atleast_basic_sanity_services']/100, row['%_of_urban_using_atleast_basic_sanity_services']/100]]
            constants = [row['%_using_atleast_basic_drinking_water_services'], row['%_using_atleast_basic_sanity_services']]
            
            try:
                # Try to solve the system using the regular solve() function
                solution = np.linalg.solve(coefficients, constants)
                rural_population_percentage = float(format(solution[0], '.2f'))
                urban_population_percentage = float(format(solution[1], '.2f'))

            except np.linalg.LinAlgError as e:
                # If the matrix is singular, use the inverse instead
                print(f"Warning: {e}")
                inverse = np.linalg.pinv(coefficients)
                solution = np.dot(inverse, constants)
                rural_population_percentage = float(format(solution[0], '.2f'))
                urban_population_percentage = float(format(solution[1], '.2f'))

            if (rural_population_percentage + urban_population_percentage) != 100:
                 # Calculate normalized values
                normalized_rural_percentage = (rural_population_percentage / (rural_population_percentage + urban_population_percentage)) * 100
                normalized_urban_percentage = (urban_population_percentage / (rural_population_percentage + urban_population_percentage)) * 100
                # Round to 2 decimal places
                normalized_rural_percentage = round(normalized_rural_percentage, 2)
                normalized_urban_percentage = round(normalized_urban_percentage, 2)
                # Assign the values to new columns
                df.at[index, '%_of_rural_population'] = normalized_rural_percentage
                df.at[index, '%_of_urban_population'] = normalized_urban_percentage
            else:
                df.at[index, '%_of_rural_population'] = rural_population_percentage
                df.at[index, '%_of_urban_population'] = urban_population_percentage
        
        # For rows with no values for '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services',
        # '%_of_rural_using_atleast_basic_sanity_services', and '%_of_urban_using_atleast_basic_sanity_services' insert 0's because they can't be calculated
        if pd.notnull(row['%_using_atleast_basic_drinking_water_services']) and pd.isnull(row['%_of_rural_using_atleast_basic_drinking_water_services']) \
            and pd.isnull(row['%_of_urban_using_atleast_basic_drinking_water_services']) and pd.notnull(row['%_using_atleast_basic_sanity_services']) \
            and pd.isnull(row['%_of_rural_using_atleast_basic_sanity_services']) and pd.isnull(row['%_of_urban_using_atleast_basic_sanity_services']) and \
            pd.notnull(row['%_of_rural_population']) and pd.notnull(row['%_of_urban_population']):

            df.at[index, '%_of_rural_using_atleast_basic_drinking_water_services'] = 0.00
            df.at[index, '%_of_urban_using_atleast_basic_drinking_water_services'] = 0.00
            df.at[index, '%_of_rural_using_atleast_basic_sanity_services'] = 0.00
            df.at[index, '%_of_urban_using_atleast_basic_sanity_services'] = 0.00

    return df        

            
df = fill_missing_percentage_population(df)

We can now see that '%_of_rural_population' and '%_of_urban_population' have been filled.

The last row, however, still has null values because '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services', '%_using_atleast_basic_sanity_services', '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services' are null and yet these are the values required by the algorithm to calculate the '%_of_rural_population' and '%_of_urban_population'

In [849]:
# Obtain rows where the country is Eritrea
eritrea = df[df['Country'] == 'Eritrea'].reset_index(drop=True)
# Display population percentages
print("One row still has null values for '%_of_rural_population' and '%_of_urban_population' because the columns whose values are used to calculate them were null")
eritrea[['Country', 'Year', '%_of_rural_population', '%_of_urban_population', 'annual_%_growth_of_rural_population', 'annual_%_growth_of_urban_population']]

One row still has null values for '%_of_rural_population' and '%_of_urban_population' because the columns whose values are used to calculate them were null


,Country,Year,%_of_rural_population,%_of_urban_population,annual_%_growth_of_rural_population,annual_%_growth_of_urban_population
0,Eritrea,2007,66.98,33.02,1.22,5.62
1,Eritrea,2008,66.00,34.00,0.71,5.12
2,Eritrea,2009,65.41,34.59,0.96,3.55
3,Eritrea,2010,64.82,35.18,0.70,3.30
4,Eritrea,2011,64.23,35.77,0.44,3.04
5,Eritrea,2012,63.46,36.54,NaN,NaN
6,Eritrea,2013,62.88,37.12,NaN,NaN
7,Eritrea,2014,62.20,37.80,NaN,NaN
8,Eritrea,2015,61.72,38.28,NaN,NaN
9,Eritrea,2016,61.15,38.85,NaN,NaN


We realised that we are working with time-series data, which means that the data contains trends or patterns which are clearly visible to the naked eye.

Mean and mode imputation were therefore not an option because they would introduce biases and affect existing trends or patterns.

We developed an algorithm that utilises regression, to predict and fill the missing values.

In [850]:
all_columns = df.columns
def fill_with_regression(original_dataframe, new_dataframe, target_column, feature_column):
    # Filter rows with available data to train the model
    df_train = new_dataframe.dropna(subset=[target_column])

    # Create a linear regression model
    model = LinearRegression()

    # Fit the model
    X_train = df_train[feature_column].to_numpy().reshape(-1, 1)
    y_train = df_train[target_column].to_numpy()
    model.fit(X_train, y_train)

    # Predict the missing values
    X_pred = new_dataframe[new_dataframe[target_column].isnull()][feature_column].to_numpy().reshape(-1, 1)
    y_pred = model.predict(X_pred)

    # Update the DataFrame with the rounded predicted values
    new_dataframe.loc[new_dataframe[target_column].isnull(), target_column] = y_pred

    # Round off to 2 decimal places
    new_dataframe[target_column] = new_dataframe[target_column].round(2)

    # merging the original dataframe with the dataframe containing filled in values 
    merged_df = pd.merge(original_dataframe, new_dataframe, on=['Country', "Country Code", 'Year'], how='left')

    exclude_columns = ['Country', 'Year', 'Country Code']

    for column in all_columns:
        if column not in exclude_columns:
            # Add data from the eritrea dataframe where it is missing in the original dataframe
            merged_df[column] = np.where(
            (merged_df[column + '_x'].isnull()) &
            (~merged_df[column + '_y'].isnull()),
            merged_df[column + '_y'],
            merged_df[column + '_x']
            )
            # Delete the unnecessary columns
            merged_df = merged_df.drop([column + '_x', column + '_y'], axis=1)

    # Calculate MSE and R-squared for the model
    mse = round(mean_squared_error(y_train, model.predict(X_train)), 2)
    r_squared = round(r2_score(y_train, model.predict(X_train)), 2)

    return merged_df, mse, r_squared, target_column

eritrea_columns = ['%_of_rural_population', '%_of_urban_population','annual_%_growth_of_rural_population', 'annual_%_growth_of_urban_population']

for column in eritrea_columns:
    df, mse, r_squared, target_column = fill_with_regression(df, eritrea, column, 'Year')
    print(f'The Mean Squared Error for the model used to predict {target_column} values is: {mse}')
    print(f'The r_squared for the model used to predict {target_column} values is: {r_squared}')
    print("")

print('All missing values have been filled')

eritrea[['Country', 'Year', '%_of_rural_population', '%_of_urban_population', 'annual_%_growth_of_rural_population', 'annual_%_growth_of_urban_population']]


The Mean Squared Error for the model used to predict %_of_rural_population values is: 0.01
The r_squared for the model used to predict %_of_rural_population values is: 1.0

The Mean Squared Error for the model used to predict %_of_urban_population values is: 0.01
The r_squared for the model used to predict %_of_urban_population values is: 1.0

The Mean Squared Error for the model used to predict annual_%_growth_of_rural_population values is: 0.02
The r_squared for the model used to predict annual_%_growth_of_rural_population values is: 0.71

The Mean Squared Error for the model used to predict annual_%_growth_of_urban_population values is: 0.11
The r_squared for the model used to predict annual_%_growth_of_urban_population values is: 0.9

All missing values have been filled


,Country,Year,%_of_rural_population,%_of_urban_population,annual_%_growth_of_rural_population,annual_%_growth_of_urban_population
0,Eritrea,2007,66.98,33.02,1.22,5.62
1,Eritrea,2008,66.00,34.00,0.71,5.12
2,Eritrea,2009,65.41,34.59,0.96,3.55
3,Eritrea,2010,64.82,35.18,0.70,3.30
4,Eritrea,2011,64.23,35.77,0.44,3.04
5,Eritrea,2012,63.46,36.54,0.33,2.03
6,Eritrea,2013,62.88,37.12,0.18,1.33
7,Eritrea,2014,62.20,37.80,0.02,0.64
8,Eritrea,2015,61.72,38.28,-0.14,-0.06
9,Eritrea,2016,61.15,38.85,-0.29,-0.76


Missing values for '%\_of_rural_population', '%\_of_rural_population', 'annual_%\_growth_of_rural_population' and 'annual\_%_growth_of_urban_population' have been filled.

We then went ahead to fill missing values for '%\_of_urban_using_atleast_basic_sanity_services', '%\_of_rural_using_atleast_basic_drinking_water_services', '%\_of_urban_using_atleast_basic_drinking_water_services', '%\_of_rural_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_using_atleast_basic_sanity_services'   

South Sudan, Eritrea and Central African Republic have rows with no data for these columns.

We again noticed trends in this data and therefore, regression imputation was used.

    

In [851]:
South_Sudan = df[df['Country'] == "South Sudan"]
South_Sudan[['Country', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']].reset_index(drop=True)

,Country,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN
1,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN
2,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN
3,South Sudan,NaN,NaN,NaN,NaN,NaN,NaN
4,South Sudan,6.63,41.09,38.68,52.01,3.95,18.81
5,South Sudan,7.31,41.09,38.21,54.05,4.16,21.50
6,South Sudan,8.04,41.07,37.67,56.13,4.36,24.31
7,South Sudan,8.80,41.01,37.07,58.24,4.57,27.26
8,South Sudan,9.60,40.93,36.40,60.40,4.78,30.33
9,South Sudan,10.44,40.81,35.67,62.59,4.99,33.53


In [852]:

null_columns = ['%_using_atleast_basic_drinking_water_services', '%_using_atleast_basic_sanity_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services',
               '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']
for column in null_columns:
    df, mse, r_squared, target_column = fill_with_regression(df, South_Sudan, column, 'Year')
    print(f'The Mean Squared Error for the model used to predict {target_column} values is: {mse}')
    print(f'The r_squared for the model used to predict {target_column} values is: {r_squared}')
    print('')

print("All specified columns with missing values have been filled ")

South_Sudan[['Country', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services', 
             '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']].reset_index(drop=True)

The Mean Squared Error for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.0
The r_squared for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.89

The Mean Squared Error for the model used to predict %_using_atleast_basic_sanity_services values is: 0.0
The r_squared for the model used to predict %_using_atleast_basic_sanity_services values is: 1.0

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.01
The r_squared for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.99

The Mean Squared Error for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 0.0
The r_squared for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 1.0

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_sanity_servic

,Country,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,South Sudan,3.40,41.44,41.38,43.37,3.12,6.45
1,South Sudan,4.19,41.37,40.74,45.51,3.33,9.46
2,South Sudan,4.97,41.30,40.11,47.64,3.54,12.47
3,South Sudan,5.75,41.23,39.48,49.78,3.74,15.48
4,South Sudan,6.63,41.09,38.68,52.01,3.95,18.81
5,South Sudan,7.31,41.09,38.21,54.05,4.16,21.50
6,South Sudan,8.04,41.07,37.67,56.13,4.36,24.31
7,South Sudan,8.80,41.01,37.07,58.24,4.57,27.26
8,South Sudan,9.60,40.93,36.40,60.40,4.78,30.33
9,South Sudan,10.44,40.81,35.67,62.59,4.99,33.53


In [853]:
# Obtain rows where the country is Eritrea
eritrea = df[df['Country'] == 'Eritrea'].reset_index(drop=True)
# Display population percentages
print("One row has null values for these columns")
eritrea[['Country', 'Year', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services',
          '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']]

One row has null values for these columns


,Country,Year,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,Eritrea,2007,10.56,49.25,29.13,90.06,3.82,24.21
1,Eritrea,2008,10.83,49.65,28.86,89.99,4.17,23.77
2,Eritrea,2009,11.02,49.81,28.60,89.93,4.52,23.33
3,Eritrea,2010,11.20,49.98,28.33,89.87,4.86,22.88
4,Eritrea,2011,11.37,50.15,28.07,89.80,5.21,22.44
5,Eritrea,2012,11.54,50.33,27.80,89.74,5.56,21.99
6,Eritrea,2013,11.64,50.71,27.80,89.74,5.56,21.99
7,Eritrea,2014,11.74,51.08,27.80,89.74,5.56,21.99
8,Eritrea,2015,11.84,51.47,27.80,89.74,5.56,21.99
9,Eritrea,2016,11.94,51.85,27.80,89.74,5.56,21.99


In [854]:
for column in null_columns:
    df, mse, r_squared, target_column = fill_with_regression(df, eritrea, column, 'Year')
    print(f'The Mean Squared Error for the model used to predict {target_column} values is: {mse}')
    print(f'The r_squared for the model used to predict {target_column} values is: {r_squared}')
    print('')
    
print("All specified columns with missing values have been filled using regression")

eritrea[['Country', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services', 
             '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']].reset_index(drop=True)

The Mean Squared Error for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.02
The r_squared for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.97

The Mean Squared Error for the model used to predict %_using_atleast_basic_sanity_services values is: 0.01
The r_squared for the model used to predict %_using_atleast_basic_sanity_services values is: 0.97

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.04
The r_squared for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.84

The Mean Squared Error for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 0.0
The r_squared for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 0.84

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_sanity_se

,Country,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,Eritrea,10.56,49.25,29.13,90.06,3.82,24.21
1,Eritrea,10.83,49.65,28.86,89.99,4.17,23.77
2,Eritrea,11.02,49.81,28.60,89.93,4.52,23.33
3,Eritrea,11.20,49.98,28.33,89.87,4.86,22.88
4,Eritrea,11.37,50.15,28.07,89.80,5.21,22.44
5,Eritrea,11.54,50.33,27.80,89.74,5.56,21.99
6,Eritrea,11.64,50.71,27.80,89.74,5.56,21.99
7,Eritrea,11.74,51.08,27.80,89.74,5.56,21.99
8,Eritrea,11.84,51.47,27.80,89.74,5.56,21.99
9,Eritrea,11.94,51.85,27.80,89.74,5.56,21.99


In [855]:
central_african_republic = df[df['Country'] == 'Central African Republic'].reset_index(drop=True)
# Display population percentages
print("One row has null values for these columns")
central_african_republic[['Country', 'Year', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services',
          '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']]

One row has null values for these columns


,Country,Year,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,Central African Republic,2007,20.55,51.10,38.18,71.87,8.76,39.50
1,Central African Republic,2008,21.37,50.07,37.31,70.44,8.82,41.42
2,Central African Republic,2009,22.21,49.04,36.43,69.01,8.88,43.34
3,Central African Republic,2010,23.07,48.01,35.55,67.59,8.94,45.26
4,Central African Republic,2011,23.94,46.98,34.65,66.16,9.00,47.18
5,Central African Republic,2012,24.83,45.95,33.75,64.74,9.05,49.10
6,Central African Republic,2013,24.94,46.04,33.75,64.74,9.05,49.10
7,Central African Republic,2014,25.06,46.13,33.75,64.74,9.05,49.10
8,Central African Republic,2015,25.18,46.23,33.75,64.74,9.05,49.10
9,Central African Republic,2016,25.32,46.33,33.75,64.74,9.05,49.10


In [856]:
for column in null_columns:
    df, mse, r_squared, target_column = fill_with_regression(df, central_african_republic, column, 'Year')
    print(f'The Mean Squared Error for the model used to predict {target_column} values is: {mse}')
    print(f'The r_squared for the model used to predict {target_column} values is: {r_squared}')
    print('')
    
print("All specified columns with missing values have been filled using regression")

central_african_republic[['Country', '%_using_atleast_basic_sanity_services', '%_using_atleast_basic_drinking_water_services', '%_of_rural_using_atleast_basic_drinking_water_services', '%_of_urban_using_atleast_basic_drinking_water_services', 
             '%_of_rural_using_atleast_basic_sanity_services', '%_of_urban_using_atleast_basic_sanity_services']].reset_index(drop=True)

The Mean Squared Error for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.65
The r_squared for the model used to predict %_using_atleast_basic_drinking_water_services values is: 0.79

The Mean Squared Error for the model used to predict %_using_atleast_basic_sanity_services values is: 0.27
The r_squared for the model used to predict %_using_atleast_basic_sanity_services values is: 0.9

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.4
The r_squared for the model used to predict %_of_rural_using_atleast_basic_drinking_water_services values is: 0.84

The Mean Squared Error for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 1.05
The r_squared for the model used to predict %_of_urban_using_atleast_basic_drinking_water_services values is: 0.84

The Mean Squared Error for the model used to predict %_of_rural_using_atleast_basic_sanity_ser

,Country,%_using_atleast_basic_sanity_services,%_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_drinking_water_services,%_of_urban_using_atleast_basic_drinking_water_services,%_of_rural_using_atleast_basic_sanity_services,%_of_urban_using_atleast_basic_sanity_services
0,Central African Republic,20.55,51.10,38.18,71.87,8.76,39.50
1,Central African Republic,21.37,50.07,37.31,70.44,8.82,41.42
2,Central African Republic,22.21,49.04,36.43,69.01,8.88,43.34
3,Central African Republic,23.07,48.01,35.55,67.59,8.94,45.26
4,Central African Republic,23.94,46.98,34.65,66.16,9.00,47.18
5,Central African Republic,24.83,45.95,33.75,64.74,9.05,49.10
6,Central African Republic,24.94,46.04,33.75,64.74,9.05,49.10
7,Central African Republic,25.06,46.13,33.75,64.74,9.05,49.10
8,Central African Republic,25.18,46.23,33.75,64.74,9.05,49.10
9,Central African Republic,25.32,46.33,33.75,64.74,9.05,49.10


In [857]:
specified_columns = ['%_of_rural_using_atleast_basic_drinking_water_services',
                    '%_of_urban_using_atleast_basic_drinking_water_services',
                    '%_of_rural_using_atleast_basic_sanity_services',
                    '%_of_urban_using_atleast_basic_sanity_services',
                    'annual_%_growth_of_rural_population',
                    'annual_%_growth_of_urban_population',
                    '%_using_atleast_basic_drinking_water_services',
                    '%_using_atleast_basic_sanity_services',
                    '%_of_rural_population',
                    '%_of_urban_population',
                    ]

percentage_of_null = (df[specified_columns].isnull().sum()/df.shape[0]*100).sort_values(ascending=False)
percentage_of_null = percentage_of_null.apply(lambda x: f'{x:.2f}%')
percentage_of_null

%_of_rural_using_atleast_basic_drinking_water_services    0.00%
%_of_urban_using_atleast_basic_drinking_water_services    0.00%
%_of_rural_using_atleast_basic_sanity_services            0.00%
%_of_urban_using_atleast_basic_sanity_services            0.00%
annual_%_growth_of_rural_population                       0.00%
annual_%_growth_of_urban_population                       0.00%
%_using_atleast_basic_drinking_water_services             0.00%
%_using_atleast_basic_sanity_services                     0.00%
%_of_rural_population                                     0.00%
%_of_urban_population                                     0.00%
dtype: object

We dropped the columns, '%_of_rural_using_safe_drinking_water_services', '%_of_urban_using_safe_drinking_water_services', '%_using_safe_sanity_services', '%_of_rural_using_safe_sanity_services', '%_of_urban_using_safe_sanity_services' because of the reasons below.

1. They have a very high percentage of missing values. This being time-series data(data with patterns or trends), most if not all of the possible imputatiom methods were not applicable. Regression imputation was not possible because there was no tangible amount of data to train the models.
Mean or mode imputation would affect underlying trends and introduce bias.

2. According to "World Bank Open Data website", The percentage of people using at least basic services encompasses both people using basic services as well as those using safely managed services. We then based our analysis on columns with "People using atleast basic services" to avoid redundancy in our analysis.

In [858]:
columns_to_drop = ['%_of_rural_using_safe_drinking_water_services',
       '%_of_urban_using_safe_drinking_water_services',
       '%_using_safe_sanity_services', '%_of_rural_using_safe_sanity_services',
       '%_of_urban_using_safe_sanity_services']

df.drop(columns=columns_to_drop, inplace=True)

Fill missing values for 'Incidences_per_1000_population_at_risk', 'Reported_cases', '%_using_IBNs', '%_of_children_under_age_5_with_fever',    '%_of_pregnant_women_using_IPT'